# CÁLCULO DE VIGA PROTENDIDA - V1307

**OBRA:** Maltaria Campos Gerais

**Cliente:** Agrária/Viero

**Data:** 29/11/2022

**Cálculo:** Gabriel Garcia

<img src="img/logoRS.jpg" alt="RS Engenharia & Arquitetura" width="200px"/>

Todas as grandezas apresentadas possuem como unidades base kN [força] e cm [comprimento].

In [3]:
import handcalcs.render
import pandas as pd
import numpy as np
from math import floor,ceil,sqrt,exp
from numpy import log, pi
from ggcp import calcpsi, linha_neutra_cp_ret

# Informações Iniciais:

## Geometria da Viga:

In [4]:
%%render
# Geometria da Viga:

h1 = 70
h0 = 54
b = 30

A_c0 = b*h0
A_c1 = b*h1

I_c0 = (b*h0**3/12)
I_c1 = (b*h1**3/12)

y_0 = h0/2
y_1 = h1/2

W_0 = I_c0/(y_0)
W_1 = I_c1/(y_1)

Vao = 728

## Número de seções para dividir o vão:
# Escolher número impar
nS = 35

<IPython.core.display.Latex object>

## Propriedades dos Materiais:

In [5]:
%%render
# Propriedades dos Materiais:

f_ck = 4
f_ctm = 0.03*(10*f_ck)**(2/3)
f_ptk = 190
f_pyd = 148.7
f_yk = 50
f_cj = 2.8

E_s = 21000
E_p = 20000

<IPython.core.display.Latex object>

In [6]:
%%render 2 short

f_c = f_ck*0.85/1.4
E_ci = 560*(f_ck*10)**(1/2)
E_cs = E_ci*(0.8+0.2*f_ck/8)

E_cit0 = (f_cj/f_ck)**(1/2)*E_ci
E_cst0 = E_cit0*(0.8+0.2*f_cj/8)

f_yd = f_yk/1.15
sigma_p0 = 0.8*f_ptk

<IPython.core.display.Latex object>

## Carregamentos:

In [7]:
%%render 4 short

Q_t0 = 0.000025*A_c0
Q_tF = 70.742/100
Q_els = 67.7/100

<IPython.core.display.Latex object>

## Esforços no Centro do Vão:

In [10]:
%%render 2 short
M_F = (Q_tF)*Vao**2*0.125
M_0 = (Q_t0)*Vao**2*0.125
V_F = (Q_tF)*Vao/2

<IPython.core.display.Latex object>

# Deteminação da Força de Protensão:

## Limites de Tensão:

In [11]:
%%render short

sigma_ct0 = -1.2*0.03*(f_cj*10)**(2/3)
sigma_cc0 = 0.7*f_cj
sigma_ctF = -1.2*0.03*(f_ck*10)**(2/3)
sigma_ccF = 0.5*f_ck

r_0 = 0.95
r_F = 0.80

<IPython.core.display.Latex object>

## Cálculo da Força de Protensão:

In [12]:
%%render short 2

e_p0 = 15
e_p1 = 23

#tempo inicial:
P_1 = (sigma_cc0+M_0/W_0)/(r_0*(1/A_c0 + e_p0/W_0)) 
P_2 = (sigma_ct0-M_0/W_0)/(r_0*(1/A_c0 - e_p0/W_0)) 

#tempo final:
P_3 = (sigma_ctF+M_F/W_1)/(r_F*(1/A_c1 + e_p1/W_1))
P_4 = (sigma_ccF-M_F/W_1)/(r_F*(1/A_c1 - e_p1/W_1))

##Adotar valores entre:
P_min = max(P_3, P_4)
P_max = min(P_1, P_2)


<IPython.core.display.Latex object>

## Escolha da Protensão:

In [14]:
%%render 2
# Força de Protensão Adotada:

P = P_max
phi_1 = 1.27
A_p1 = 1.01

## Número de cordoalhas para adotar:
n = floor(P/(A_p1*sigma_p0))

<IPython.core.display.Latex object>

## Ancoragem das Cordoalhas:

### Comprimento de Ancoragem Básico:

In [15]:
%%render short
f_pyd = 0.9* f_ptk / 1.15

eta_p1 = 1.2

# Valores de Eta_p1:
#     
#     1.0 para fios lisos
#     1.2 para cordoalhas de 3 e 7 fios
#     1.5 para fios dentados

eta_p2 = 1.0

# Valores de Eta_p2:
#     
#     1.0 para situação de boa aderência
#     0.7 para situação de má aderência

f_ctd = (0.7*0.03*(f_cj*10)**(2/3))/1.4
f_bpd = eta_p1 * eta_p2 * f_ctd

l_bp = (7*phi_1)/36 * f_pyd/f_bpd

<IPython.core.display.Latex object>

### Cálculo do comprimento de Transferência:

In [16]:
%%render short

## para fios lisos ou dentados:
l_bpt1 = 0.7 * l_bp * sigma_p0/f_pyd

## para cordoalhas de 3 e 7 fios:
l_bpt2 = 0.5 * l_bp * sigma_p0/f_pyd

## para transferência não gradual, os valores devem ser multiplicados por 1.25
## valor adotado:

l_bpt = 1.25 * l_bpt2

l_p = (h1**2 + (0.6*l_bpt)**2)**(0.5)


<IPython.core.display.Latex object>

#  Esforços nas Seções:

## Momento Fletor e Cortante:

In [17]:
esforcos = []
esforcos_t0 = []
for i in range(3):
    esforcos.append( [0]*(nS) )
    esforcos_t0.append( [0]*(nS) )
    
# primeiro vetor contem X (cm)
# segundo vetor contem Momentos (kNcm)
# terceiro vetor contém cortante (kN)

for i in range(nS):
    esforcos[0][i] = round(i*(Vao/(nS-1)),1)
    esforcos[1][i] = round( 0.5*Q_tF*esforcos[0][i]*(Vao-esforcos[0][i]) ,1)
    esforcos[2][i] = round( 0.5*Q_tF*(Vao-2*esforcos[0][i]), 1 )
    esforcos_t0[0][i] = round(i*(Vao/(nS-1)),1)
    esforcos_t0[1][i] = round( 0.5*Q_t0*esforcos[0][i]*(Vao-esforcos[0][i]) ,1)
    esforcos_t0[2][i] = round( 0.5*Q_t0*(Vao-2*esforcos[0][i]), 1 )
    
    
tabela = pd.DataFrame(esforcos)
tabela = tabela.transpose()
tabela.rename(columns={0:'Pos (cm)',1:'M (kNcm)',2:'V (kN)'},inplace = True)
display(tabela)

,Pos (cm),M (kNcm),V (kN)
0,0.0,0.0,257.5
1,21.4,5348.5,242.4
2,42.8,10373.1,227.2
3,64.2,15073.7,212.1
4,85.6,19450.3,196.9
5,107.1,23521.1,181.7
6,128.5,27248.3,166.6
7,149.9,30651.5,151.5
8,171.3,33730.7,136.3
9,192.7,36486.0,121.2


# Escalonamento da Protensão:

## Limites de Tensão na Seção S1:

São testados os limites de tensão para t0:

Limitação pela borda superior:

Momento adotado: momento de peso próprio na seção onde passa a valer a tensão máxima das coordoalhas

In [18]:
%%render short

M_s1 = 0.5*Q_t0*l_bpt*(Vao-l_bpt)
P_1 = (sigma_cc0+M_s1/W_0)/(r_0*(1/A_c0 + e_p0/W_0))
P_2 = (sigma_ct0-M_s1/W_0)/(r_0*(1/A_c0 - e_p0/W_0))


<IPython.core.display.Latex object>

In [19]:
%%render 1

P_s1 = min(P_1 , P_2)

n_s1 = floor(P_s1/(A_p1*sigma_p0))


<IPython.core.display.Latex object>

## Escolha do Número de Cordoalhas:

In [22]:
%%render

# Vetor com o número de cordoalhas em cada ponto de escalonamento:

qntCordoalhas = [6,8]

# Vetor com a excentricidade da protensão em cada ponto de escalonamento:

e_p_i = [15.3,14.5]

<IPython.core.display.Latex object>

# Esforços Devidos À Protensão:

In [23]:
A_p = []

for i in range(len(qntCordoalhas)):
    A_p.append(qntCordoalhas[i] * A_p1)

W_cp = []

for i in range(len(qntCordoalhas)):
    W_cp.append(round(I_c1/e_p_i[i],0))
    
alpha_p0 = E_p / E_cst0

K_p0 = []
for i in range(len(qntCordoalhas)):
    K_p0.append(round(alpha_p0*A_p[i]*(1/A_c1+e_p_i[i]/W_cp[i]),4))
    
sigma_p = []
for i in range(len(qntCordoalhas)):
    sigma_p.append(round(sigma_p0/(1+K_p0[i]),2))
    
N_p = []
for i in range(len(qntCordoalhas)):
    N_p.append(round(-sigma_p[i]*A_p[i],1))
    
M_p = []
for i in range(len(qntCordoalhas)):
    M_p.append(round(N_p[i]*e_p_i[i],1))
    
tabela1 = pd.DataFrame([A_p, W_cp, K_p0, sigma_p, N_p, M_p])
tabela1 = tabela1.transpose()
tabela1.rename(columns={0:'Ap',1:'Wcp',2:'Kp0',3:'Sigma,p',4:'Np',5:'Mp'},inplace = True)
display(tabela1)

,Ap,Wcp,Kp0,"Sigma,p",Np,Mp
0,6.06,56046.0,0.0352,146.83,-889.8,-13613.9
1,8.08,59138.0,0.0452,145.43,-1175.1,-17038.9


# Escalonamento da Força de Protensão:

In [24]:
M_g = []
x1 = []
x2 = []
x1_anc = []
x2_anc = []

for i in range(len(qntCordoalhas)):
    M_g.append(round( W_0*( (N_p[i]/A_c0) -(M_p[i]/W_0) + sigma_ct0),2) )
    #print("M_g["+str(i)+"]:",M_g[i])
        
for i in range(len(qntCordoalhas)):
    # O erro acontece no caso da raiz de delta retornar um número complexo
    try:
        c = 2*M_g[i]/Q_t0
        delta = (-Vao)**2 - 4*c
        
        #print("Delta "+str(i)+":",round(delta,2))
        x1.append(round( (Vao-(delta**0.5))/2  ,2))
        x2.append(round( (Vao+(delta**0.5))/2  ,2))
    except:
        print("##   ATENÇÃO!   ##")
        raise Exception("ERRO: ESCALONAR MELHOR A PROTENSÃO!")    

for i in range(len(qntCordoalhas)):
    x1_anc.append(x1[i]-round(l_bpt,2))
    x2_anc.append(x2[i]+round(l_bpt,2))
    if x1_anc[i] < 0:
        x1_anc[i] = 0
    if x2_anc[i] > Vao:
        x2_anc[i] = Vao
    
tabela2 = pd.DataFrame([M_g, x1,x2, x1_anc,x2_anc])
tabela2 = tabela2.transpose()
tabela2.rename(columns={0:'Mg',1:'x1',2:'x2',3:'x1-lbpt',4:'x2+lbpt'},inplace = True)
display(tabela2)

,Mg,x1,x2,x1-lbpt,x2+lbpt
0,765.85,56.30,671.70,0.0,728.0
1,1623.15,135.22,592.78,0.0,728.0


# Perdas Progressivas da Força de Protensão:

## Fluência do Concreto:

### Fluência Rápida Irreversível

In [25]:
%%render params

# VER ITEM 12.3.3 da NBR6118:2014
# Valores para adotar:
# s = 0.38 para concreto de cimento CP-III e CP-IV
# s = 0.25 para concreto de cimento CP-I e CP-II
# s = 0.20 para concreto de cimento CP-V
s = 0.20
t_0 = 10
t_F = 18250


beta_0 = exp(s*(1-(28/t_0)**0.5))
beta_F = exp(s*(1-(28/t_F)**0.5))

phi_a = 0.8*(1 - (beta_0/beta_F))

<IPython.core.display.Latex object>

In [26]:
if s == 0.38:
    print("Adotado s = 0.38 para concreto de cimento CP-III ou CP-IV")
elif s == 0.25:
    print("Adotado s = 0.25 para concreto de cimento CP-I ou CP-II")
else:
    print("Adotado s = 0.20 para concreto de cimento CP-V (ARI) ")

Adotado s = 0.20 para concreto de cimento CP-V (ARI) 


### Fluência Lenta Irreversível:

In [27]:
%%render params 2

U = 55 #umidade média do ar

phi_c1 = 4.45 - 0.035*U # para abatimentos entre 5cm e 9cm

gamma = 1 + exp(-7.8 + 0.1 * U)

u_ar = ( 2*h1 + 2*b )

h_fic = gamma*(2*A_c1)/u_ar

phi_c2 = (42+h_fic)/(20+h_fic)

phi_fF = phi_c1*phi_c2

h_fm = h_fic/100 # espessura fictícia em metros

A_beta_f = 42*h_fm**3 - 350*h_fm**2 + 588*h_fm + 113
B_beta_f = 768*h_fm**3 - 3060*h_fm**2 + 3234*h_fm - 23
C_beta_f = -200*h_fm**3 + 13*h_fm**2 + 1090*h_fm + 183
D_beta_f = 7579*h_fm**3 - 31916*h_fm**2 + 35343*h_fm + 1931

beta_f_t0 = (t_0**2 + A_beta_f*t_0 + B_beta_f )/(t_0**2 + C_beta_f*t_0 + D_beta_f)
beta_f_tF = (t_F**2 + A_beta_f*t_F + B_beta_f )/(t_F**2 + C_beta_f*t_F + D_beta_f) 

<IPython.core.display.Latex object>

### Fluência Lenta Reversível:

In [29]:
%%render params

phi_dF = 0.4

beta_d = (t_F-t_0+20)/(t_F-t_0+70)

<IPython.core.display.Latex object>

### Fluência Total:

In [30]:
%%render 2 params
phi_t = phi_a + phi_fF*(beta_f_tF - beta_f_t0) + phi_dF * beta_d

<IPython.core.display.Latex object>

### Retração do Concreto:

In [31]:
%%render 2 params

epsilon_1s = (-8.09 + U/15 - U**2/2284 + U**3/133765 - U**4/7608150)*10**-4

epsilon_2s = (33+2*h_fic)/(20.8 + 3*h_fic)

A_beta_s = 40  
B_beta_s = 116*h_fm**3 - 282*h_fm**2 + 220*h_fm - 4.8  
C_beta_s = 2.5*h_fm**3 - 8.8*h_fm + 40.7  
D_beta_s = -75*h_fm**3 + 585*h_fm**2 + 496*h_fm - 6.8  
E_beta_s = -169*h_fm**4 + 88*h_fm**3 + 584*h_fm**2 - 39*h_fm + 0.8  

beta_s_t0 = ((t_0/100)**3 + A_beta_s*(t_0/100)**2 +  B_beta_s*(t_0/100))/((t_0/100)**3 + C_beta_s*(t_0/100)**2 +  D_beta_s*(t_0/100) + E_beta_s)

beta_s_tF = ((t_F/100)**3 + A_beta_s*(t_F/100)**2 +  B_beta_s*(t_F/100))/((t_F/100)**3 + C_beta_s*(t_F/100)**2 +  D_beta_s*(t_F/100) + E_beta_s)

epsilon_cs = epsilon_1s*epsilon_2s*(beta_s_tF - beta_s_t0)


<IPython.core.display.Latex object>

## Relaxação do Aço de Protensão:

In [32]:
relax = []
for i in range(7):
    relax.append([0]*nS)

for i in range (nS):
    relax[0][i] = int(i+1)
    relax[1][i] = esforcos[0][i] #posição x da seção
    for j in range(len(qntCordoalhas)-1,-1,-1):
        if(relax[1][i] >= x1_anc[j] and relax[1][i] <= x2_anc[j]):
            relax[2][i] = round(sigma_p[j],2)
            break
    relax[3][i] = round(relax[2][i]/f_ptk,3)
    relax[4][i] = round(calcpsi(relax[3][i]),3)
    relax[5][i] = round(relax[4][i]*2.5,3)
    relax[6][i] = round(-log(1-relax[5][i]),3)
    
    
tabela3 = pd.DataFrame(relax)
tabela3 = tabela3.transpose()
tabela3.rename(columns={0:'Seção',1:'Pos',2:'Sigma,p',3:'Sigma,p/fptk',4:'\u03A81000',5:'\u03A8,inf',6:'X,inf'},inplace = True)
display(tabela3)

,Seção,Pos,"Sigma,p","Sigma,p/fptk",Ψ1000,"Ψ,inf","X,inf"
0,1.0,0.0,145.43,0.765,0.032,0.08,0.083
1,2.0,21.4,145.43,0.765,0.032,0.08,0.083
2,3.0,42.8,145.43,0.765,0.032,0.08,0.083
3,4.0,64.2,145.43,0.765,0.032,0.08,0.083
4,5.0,85.6,145.43,0.765,0.032,0.08,0.083
5,6.0,107.1,145.43,0.765,0.032,0.08,0.083
6,7.0,128.5,145.43,0.765,0.032,0.08,0.083
7,8.0,149.9,145.43,0.765,0.032,0.08,0.083
8,9.0,171.3,145.43,0.765,0.032,0.08,0.083
9,10.0,192.7,145.43,0.765,0.032,0.08,0.083


## Cálculo das Perdas Progressivas:

In [33]:
progressivas = []

for i in range(4):
    progressivas.append([0] * nS)
    
alpha_p = E_p/E_cs

delta_Np_max = 0
delta_Mp_max = 0
    
for i in range(nS):
    pos_i = relax[1][i]
    momento = 0.5*Q_t0*pos_i*(Vao-pos_i)
    #momento = esforcos[1][i]
    #print("Momento seção "+str(i)+": "+str(int(momento)))
    for j in range(len(qntCordoalhas)-1,-1,-1):
        if(relax[1][i] >= x1_anc[j] and relax[1][i] <= x2_anc[j]):
            sigma_pi = sigma_p[j]
            Npi = N_p[j]
            Mpi = M_p[j]
            Wcpi = W_cp[j]
            e_pp = e_p_i[j]
            A_pp = A_p[j]
            break
        else:
            sigma_pi = 0
            Npi = 0
            Mpi = 0
            Wcpi = 0
            e_pp = 0
            A_pp = 0
            
    if (Npi != 0):
        progressivas[0][i] = round(Npi/A_c1 + Mpi/Wcpi + momento/Wcpi,2)
        K_p = alpha_p*A_pp*(1/A_c1 + e_pp/Wcpi)
        progressivas[1][i] = round( (( progressivas[0][i]*phi_t/E_cs + epsilon_cs - sigma_pi/E_p*relax[6][i] ) / (1 + relax[6][i] + K_p*(1+0.5*phi_t)) )*E_p,2)
        progressivas[2][i] = -round(progressivas[1][i]*A_pp  ,2)
        progressivas[3][i] = round(progressivas[2][i]*e_pp ,1)
        if(progressivas[2][i] > delta_Np_max):
            delta_Np_max = progressivas[2][i]
            delta_Mp_max = progressivas[3][i]
    else:
        progressivas[0][i] = 0
        progressivas[1][i] = 0
        progressivas[2][i] = 0
        progressivas[3][i] = 0
    
    

tabela4 = pd.DataFrame(progressivas)
tabela4 = tabela4.transpose()
tabela4.rename(columns={0:'Sigma,c,t0',1:'\u0394Sigma,p',2:'\u0394Np',3:'\u0394Mp'},inplace = True)
display(tabela4)

,"Sigma,c,t0","ΔSigma,p",ΔNp,ΔMp
0,-0.85,-33.72,272.46,3950.7
1,-0.84,-33.53,270.92,3928.3
2,-0.84,-33.53,270.92,3928.3
3,-0.83,-33.34,269.39,3906.2
4,-0.83,-33.34,269.39,3906.2
5,-0.82,-33.16,267.93,3885.0
6,-0.82,-33.16,267.93,3885.0
7,-0.82,-33.16,267.93,3885.0
8,-0.82,-33.16,267.93,3885.0
9,-0.81,-32.97,266.40,3862.8


# Tensões Normais nas Bordas:

## Bordas Superiores:

In [34]:
tensaosup = []

for i in range(3):
    tensaosup.append([0]*nS)
    
for i in range(nS):
    tensaosup[0][i] = relax[1][i]
    for j in range(len(qntCordoalhas)-1,-1,-1):
        if(relax[1][i] >= x1_anc[j] and relax[1][i] <= x2_anc[j]):
            sigma_pi = sigma_p[j]
            Npi = N_p[j]
            Mpi = M_p[j]
            Wcpi = W_cp[j]
            e_pp = e_p_i[j]
            A_pp = A_p[j]
            break
    Npx = Npi + progressivas[2][i]
    Mpx = Mpi + progressivas[3][i]
    tensaosup[1][i] = round( -abs(Npx/A_c1) + abs(Mpx/W_1) - abs(esforcos[1][i]/W_1) ,3)
    if tensaosup[1][i] > -sigma_ccF:
        tensaosup[2][i] = "OK"
    else:
        tensaosup[2][i] = "--> NAO OK <--"

tabela5 = pd.DataFrame(tensaosup)
tabela5 = tabela5.transpose()
tabela5.rename(columns={0:'Pos (cm)',1:'Sigma,s',2:'OK'},inplace = True)
display(tabela5)

,Pos (cm),"Sigma,s",OK
0,0.0,0.104,OK
1,21.4,-0.114,OK
2,42.8,-0.319,OK
3,64.2,-0.511,OK
4,85.6,-0.689,OK
5,107.1,-0.855,OK
6,128.5,-1.007,OK
7,149.9,-1.146,OK
8,171.3,-1.272,OK
9,192.7,-1.384,OK


## Bordas Inferiores:

In [35]:
tensaoinf = []

for i in range(3):
    tensaoinf.append([0]*nS)
    
for i in range(nS):
    tensaoinf[0][i] = relax[1][i]
    for j in range(len(qntCordoalhas)-1,-1,-1):
        if(relax[1][i] >= x1_anc[j] and relax[1][i] <= x2_anc[j]):
            sigma_pi = sigma_p[j]
            Npi = N_p[j]
            Mpi = M_p[j]
            Wcpi = W_cp[j]
            e_pp = e_p_i[j]
            A_pp = A_p[j]
            break
    Npx = Npi + progressivas[2][i]
    Mpx = Mpi + progressivas[3][i]
    tensaoinf[1][i] = round( -abs(Npx/A_c1) - abs(Mpx/W_1) + abs(esforcos[1][i]/W_1) ,3)
    if tensaoinf[1][i] < -sigma_ctF:
        tensaoinf[2][i] = "OK"
    else:
        tensaoinf[2][i] = "--> NAO OK <--"
    
tabela6 = pd.DataFrame(tensaoinf)
tabela6 = tabela6.transpose()
tabela6.rename(columns={0:'Pos (cm)',1:'Sigma,i',2:'OK'},inplace = True)
display(tabela6)

,Pos (cm),"Sigma,i",OK
0,0.0,-0.964,OK
1,21.4,-0.747,OK
2,42.8,-0.542,OK
3,64.2,-0.352,OK
4,85.6,-0.173,OK
5,107.1,-0.009,OK
6,128.5,0.143,OK
7,149.9,0.282,OK
8,171.3,0.408,OK
9,192.7,0.519,--> NAO OK <--


# Armaduras Passivas:

## Armadura da Cunha de Tração:

In [36]:
sigma_cs0 = -10000
pos = -1
for i in range (0,nS):
    if sigma_cs0 < tensaosup[1][i]:
        sigma_cs0 = tensaosup[1][i]
        pos = i

sigma_ci0 = tensaoinf[1][pos]

print("Seção crítica de cálculo: seção",pos)

Seção crítica de cálculo: seção 0


In [37]:
%%render 2

x = sigma_cs0/(sigma_cs0 - sigma_ci0) * h0

S_x = b*x**2/2

R_ct = sigma_cs0/x*S_x

A_st = R_ct/25.00

phi_st = 1.6

N_st = ceil(A_st/((phi_st/2)**2*pi))

<IPython.core.display.Latex object>

## Pré-Alongamento de Leito:

In [38]:
prealong = []

for i in range(5):
    prealong.append([0] * nS)
    
for i in range(nS):
    for j in range(len(qntCordoalhas)-1,-1,-1):
        if(relax[1][i] >= x1_anc[j] and relax[1][i] <= x2_anc[j]):
                sigma_pw = sigma_p[j]
                Npw = N_p[j]
                Mpw = M_p[j]
                Wcpw = W_cp[j]
                break
    prealong[0][i] = Npw + progressivas[2][i]
    prealong[1][i] = Mpw + progressivas[3][i]
    prealong[2][i] = round(prealong[0][i]/A_c1 + prealong[1][i]/Wcpw ,2)
    prealong[3][i] = round( sigma_pw + progressivas[1][i] - alpha_p*prealong[2][i] ,2)
    prealong[4][i] = round(prealong[3][i]/E_p,5)

tabela7 = pd.DataFrame(prealong)
tabela7 = tabela7.transpose()
tabela7.rename(columns={0:'Np,F',1:'Mp,F',2:'Sigma,cpF', 3:'Sigma,PN',4:'e,pn'},inplace = True)
display(tabela7)

,"Np,F","Mp,F","Sigma,cpF","Sigma,PN","e,pn"
0,-902.64,-13088.2,-0.65,115.79,0.00579
1,-904.18,-13110.6,-0.65,115.98,0.00580
2,-904.18,-13110.6,-0.65,115.98,0.00580
3,-905.71,-13132.7,-0.65,116.17,0.00581
4,-905.71,-13132.7,-0.65,116.17,0.00581
5,-907.17,-13153.9,-0.65,116.35,0.00582
6,-907.17,-13153.9,-0.65,116.35,0.00582
7,-907.17,-13153.9,-0.65,116.35,0.00582
8,-907.17,-13153.9,-0.65,116.35,0.00582
9,-908.70,-13176.1,-0.66,116.60,0.00583


## Armadura Passiva de Flexão:

![formulario](img/formulario.png)

In [39]:
d_p = [] # braço de alavanca da armadura ativa
d_s = [] # braço de alavanca da armadura passiva

for i in range(len(qntCordoalhas)):
    d_p.append(e_p_i[i] + h1/2)
    d_s.append(h1-5) # estimativa para ds
    
A_s = []
for i in range(8):
    A_s.append([0] * nS)
    
for i in range(nS):
    for j in range(len(qntCordoalhas)-1,-1,-1):
        if(relax[1][i] >= x1_anc[j] and relax[1][i] <= x2_anc[j]):
            sigma_pm = sigma_p[j]
            Npm = N_p[j]
            Mpm = M_p[j]
            Wcpm = W_cp[j]
            A_pm = A_p[j]
            d_sm = d_s[j]
            d_pm = d_p[j]
            break
    Md = esforcos[1][i]*1.4
    retorno = linha_neutra_cp_ret(b,h1,f_c,A_pm,prealong[4][i],d_pm,d_sm,Md)
    A_s[0][i] = int(Md)
    A_s[1][i] = retorno[0]
    A_s[2][i] = round(retorno[0]/d_sm,3)
    A_s[3][i] = retorno[2]
    A_s[5][i] = retorno[1]
    A_s[4][i] = retorno[4]
    A_s[6][i] = retorno[5]
    A_s[7][i] = round((0.8*b*f_c*retorno[0]-retorno[3])/retorno[4],2)
    

tabela8 = pd.DataFrame(A_s)
tabela8 = tabela8.transpose()
tabela8.rename(columns={0:'Md', 1:'x',2:'x/d',3:'eps,s',4:'sigma,s',5:'eps,p',6:'sigma,p',7:'As'},inplace = True)
display(tabela8)

,Md,x,x/d,"eps,s","sigma,s","eps,p","sigma,p",As
0,0.0,5.07,0.078,0.04134,43.48,0.03586,148.7,-20.84
1,7487.0,7.21,0.111,0.02804,43.48,0.02574,148.7,-17.97
2,14522.0,9.28,0.143,0.02102,43.48,0.02039,148.7,-15.19
3,21103.0,11.27,0.173,0.01669,43.48,0.01711,148.7,-12.53
4,27230.0,13.17,0.203,0.01377,43.48,0.01488,148.7,-9.98
5,32929.0,14.99,0.231,0.01168,43.48,0.01330,148.7,-7.54
6,38147.0,16.70,0.257,0.01012,43.48,0.01211,148.7,-5.25
7,42912.0,18.30,0.282,0.00893,43.48,0.01120,148.7,-3.10
8,47222.0,19.79,0.304,0.00800,43.48,0.01049,148.7,-1.10
9,51080.0,21.15,0.325,0.00726,43.48,0.00994,148.7,0.72


# Dimensionamento ao Esforço Cortante:

## Verificação da Diagonal Comprimida:

In [40]:
%%render

V_d = V_F * 1.4
tau_wd = V_d/((b)*d_sm)
tau_wu = 0.27*(1-f_ck/25)*f_ck/1.4


<IPython.core.display.Latex object>

In [41]:
if tau_wu > tau_wd:
    print("OK!")
else:
    print("NÃO OK!")

OK!


## Cálculo da Armadura de Corte:

### Armadura Mínima:

In [42]:
%%render short

rho_min = 0.2*f_ctm/f_yk
A_sws = rho_min*b

s = 15

A_sw_min = A_sws*s

nr = 2

phi_min = 2*(A_sw_min/(pi*nr))**0.5


<IPython.core.display.Latex object>

### Armadura Calculada:

In [44]:
i = int((nS+1)/2)
M_pf = prealong[1][i]
N_pf = prealong[0][i]

In [45]:
%%render

sigma_ciF = N_pf/A_c1 + M_pf/W_1
M_0 = -sigma_ciF*W_1
M_d_max = 1.4*M_F

V_c0 = 0.6*f_ctd*b*d_sm
V_c = (1+M_0/M_d_max)*V_c0

V_sw = 1.4*V_F - V_c

A_sws = V_sw/(0.9*d_sm*f_yd)

s = 20
nr = 2
phi_min = 2*(A_sws/(pi*nr))**0.5

<IPython.core.display.Latex object>

## Armadura de Introdução da Força de Protensão:

In [46]:
# VERIFICAR SE ESTA É MESMO A FORÇA DE PROTENSÃO INSERIDA NO APOIO

N_p1 = -N_p[0]

In [47]:
%%render

A_swi = (0.25*N_p1)/25.00

<IPython.core.display.Latex object>

Esta armadura deve ser distribuída num trecho equivalente à 70% do comprimento de regularização.

In [48]:
%%render

l_d = l_p*0.7

<IPython.core.display.Latex object>

# Cálculo da Flecha:

In [49]:
delta_p = []

delta_pF = 0

for i in range(len(qntCordoalhas)):
    delta_p.append([0] * 3)
    delta_p[i][0] = A_p[i]
    delta_p[i][1] = x1_anc[i]
    
    if(i != 0):
        delta_p[i][0] = delta_p[i][0] - delta_p[i-1][0]
        Momen = M_p[i] - M_p[i-1]
    else:
        Momen = M_p[i]
    
    if(delta_p[i][1] == 0):
        delta_p[i][2] = round(Momen*Vao**2/(8*E_cs*I_c1),2)
    else:
        delta_p[i][1] = (x1_anc[i]+x1[i])/2
        delta_p[i][2] = round( (Momen*(Vao**2-3*delta_p[i][1]**2-(Vao/2)**2)*(Vao/2))
                             /(6*E_cs*I_c1*Vao) ,2)
        
for i in range(len(qntCordoalhas)):
    delta_pF += delta_p[i][2]
    
tabela9 = pd.DataFrame(delta_p)
tabela9.rename(columns={0:'Ap',1:'b',2:'Delta'},inplace = True)

display(tabela9)

N_pp = N_p[len(qntCordoalhas)-1]
M_pp = M_p[len(qntCordoalhas)-1]

if (delta_p[0][0]) < 0:
    raise Exception("ERRO")

,Ap,b,Delta
0,6.06,0,-0.33
1,2.02,0,-0.08


In [50]:
%%render
delta_pF

<IPython.core.display.Latex object>

## Momento de Fissuração:


In [51]:
%%render 2

# COEFICIENTE ALPHA T:
# alpha_t = 1.2 para seções T ou duplo T
# alpha_t = 1.3 para seções I ou T invertido
# alpha_t = 1.5 para seções retangulares

alpha_t = 1.5

M_r = (alpha_t*f_ctm - N_pp/A_c1)* I_c1 / y_1 - M_pp
M_a = Q_els*0.125*Vao**2

<IPython.core.display.Latex object>

## Inércia no Estádio II:

In [52]:
%%render 2
# --------------------------------
# --------------------------------
# --------------------------------
# --------------------------------
I_II = 6.25 * 10**5 
# É NECESSÁRIO INFORMAR ESTE VALOR
# --------------------------------
# --------------------------------
# --------------------------------
# --------------------------------

<IPython.core.display.Latex object>

## Inércia Equivalente:

In [53]:
%%render

if (M_a > M_r):
    I_eq = (M_r/M_a)**3*I_c1 + (1-(M_r/M_a)**3)*I_II

if (M_a < M_r):
    I_eq = I_c1

<IPython.core.display.Latex object>

## Flecha Diferida Final:

In [54]:
%%render 2

delta_i = (delta_pF + 5/384*(Q_els*Vao**4)/(E_cs*I_eq))*(1+phi_t)

<IPython.core.display.Latex object>